In [1]:
import numpy as np
import openai
import pandas as pd
import os
import sys
import time

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, 'data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'amazon-beauty/large_merged_data.csv')
print(f'Data path: {data_path}')

# zero shot save path
ZERO_SHOT_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/output/title_large_predictions_zero_shot.csv')
print(f'Zero shot save path: {ZERO_SHOT_SAVE_PATH}')

# few shot save path
FEW_SHOT_1_OBS_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/output/title_large_1_test_predictions_few_shot.csv')
print(f'Few shot save path: {FEW_SHOT_1_OBS_SAVE_PATH}')

Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/large_merged_data.csv
Zero shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/output/title_large_predictions_zero_shot.csv
Few shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/output/title_large_1_test_predictions_few_shot.csv


# Data Overview

In [3]:
# Read the data
data = pd.read_csv(data_path)

# get statistic and first few data of NUM_SAMPLES rows
data.info()
data.head(NUM_EXAMPLES)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9767 entries, 0 to 9766
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rating          9767 non-null   float64
 1   verified        9767 non-null   bool   
 2   reviewTime      9767 non-null   object 
 3   reviewerID      9767 non-null   object 
 4   asin            9767 non-null   object 
 5   reviewerName    9767 non-null   object 
 6   reviewText      9759 non-null   object 
 7   summary         9759 non-null   object 
 8   unixReviewTime  9767 non-null   object 
 9   vote            1487 non-null   object 
 10  style           6768 non-null   object 
 11  category        9767 non-null   object 
 12  tech1           2 non-null      object 
 13  description     9767 non-null   object 
 14  fit             0 non-null      float64
 15  title           9767 non-null   object 
 16  also_buy        9767 non-null   object 
 17  tech2           0 non-null      f

,rating,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
0,1.0,False,2015-08-25,A2RYSCZOPEXOCQ,9790787006,The Cat Next Door,"I use a lot of perfume, I go through a new bot...",This is not going to be my favorite scent.,2015-08-25,NaN,...,NaN,Jenna Jameson,[],298.0,"['B00357FTX8', 'B01NBID7FJ', 'B0017JT658']","{'Shipping Weight:': '12.8 ounces (', 'ASIN: '...",All Beauty,NaN,NaN,13.85
1,5.0,False,2001-06-08,A141OPVE376YFI,B000050B65,Paul G.,"First, a little background. I've switched bet...","Finally, a razor that lives up to the ads",2001-06-08,81,...,NaN,Norelco,[],2.0,"['B01B1O9DOM', 'B00JITDVD2', 'B01KXV16DK', 'B0...",{},All Beauty,NaN,NaN,NaN
2,5.0,False,2001-06-08,A141OPVE376YFI,B000050B65,Paul G.,"First, a little background. I've switched bet...","Finally, a razor that lives up to the ads",2001-06-08,81,...,NaN,Norelco,[],2.0,"['B01B1O9DOM', 'B00JITDVD2', 'B01KXV16DK', 'B0...",{},All Beauty,NaN,NaN,NaN
3,5.0,True,2008-07-25,A1TVTDKNMSQ7XU,B000050B6B,Grandpa Pipes,I've had many Norelco razors in my 50 years of...,Just like new.....,2008-07-25,NaN,...,NaN,Philips Norelco,[],148.0,"['B001IA0PCY', 'B00196W5S4', 'B004URZADG', 'B0...",{'\n Product Dimensions: \n ': '5.1 x 0....,All Beauty,NaN,NaN,64.50
4,5.0,True,2008-07-25,A1TVTDKNMSQ7XU,B000050B6B,Grandpa Pipes,I've had many Norelco razors in my 50 years of...,Just like new.....,2008-07-25,NaN,...,NaN,Philips Norelco,[],148.0,"['B001IA0PCY', 'B00196W5S4', 'B004URZADG', 'B0...",{'\n Product Dimensions: \n ': '5.1 x 0....,All Beauty,NaN,NaN,64.50


# Zero-shot (OpenAI API)

In [ ]:
%%time

predict_ratings_zero_shot_and_save(data,
                                       columns_for_prediction=['title'],
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=ZERO_SHOT_SAVE_PATH)

In [ ]:
import pandas as pd

# Read the data
data = pd.read_csv(ZERO_SHOT_SAVE_PATH)

# Display the original data types
# print("Original Data Types:")
# print(data.dtypes)
# print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
data['is_rating_float'] = pd.to_numeric(data['zero_shot_predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = data[data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
# %%time


# # Re-run zero-shot predictions for failed cases
# zero_shot_data = pd.read_csv(ZERO_SHOT_SAVE_PATH)
# # Attempt to convert ratings to float and add a flag for conversion failure
# zero_shot_data['is_rating_float'] = pd.to_numeric(zero_shot_data['zero_shot_predicted_rating'], errors='coerce').notna()
# failed_zero_shot_indices = zero_shot_data[zero_shot_data['is_rating_float'] == False].index
# zero_shot_data = rerun_failed_predictions(
#     zero_shot_data, 
#     failed_zero_shot_indices, 
#     predict_ratings_zero_shot_and_save, 
#     ZERO_SHOT_SAVE_PATH,
#     columns_for_prediction=['title'],
#     user_column_name='reviewerID',
#     pause_every_n_users=PAUSE_EVERY_N_USERS,
#     sleep_time=SLEEP_TIME
# )

In [ ]:
evaluate_model_predictions_rmse_mae(
    data_path=ZERO_SHOT_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='rating',
    predicted_ratings_column='zero_shot_predicted_rating'
)


# Few-shot (OpenAI API)


+ For each user, we'll use 4 of their ratings as training data to predict ratings for the rest of their products. Finally, we'll evaluate the predictions against the actual ratings to calculate the overall RMSE and MAE.

+ The rating_history_str now includes both the title and the review text for each of the training data rows

# 1 observation per reviewer - Few-shot OpenAI

In [4]:
%%time

predict_ratings_few_shot_and_save(data,
                                      columns_for_training=['title'],
                                       columns_for_prediction=['title'],
                                       obs_per_user=1,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=FEW_SHOT_1_OBS_SAVE_PATH)


Constructed Prompt for few-shot approach:

The prompt:
**********
How will user rate this title: Caboodles Heart Throb Long Tapered Tote, Black Diamond, 1.12 Pound? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.

Here is user rating history:
* title: Kordon Oasis (Novalek) Bell Bottle 8oz - Rating: 5.0 stars
* title: Jenna Jameson Heartbreaker Perfume for women 3.4 oz Eau De Parfum Spray - Rating: 1.0 stars
* title: Kordon Oasis (Novalek) Bell Bottle 8oz - Rating: 5.0 stars
* title: OZNaturals Anti Aging Retinol Serum -The Most Effective Anti Wrinkle Serum Contains Professional Strength Retinol+ Astaxanthin+ Vitamin E - Get The Dramatic Youthful Results You&rsquo;ve Been Looking For - Rating: 4.0 stars

Based on the above information, please predict user's rating for the product: (1 being lowest and 5 being highest, The output should be like: (x stars, xx%), do not explain the reason.)
**********



Syst

In [ ]:
import pandas as pd

# Read the data
data = pd.read_csv(FEW_SHOT_1_OBS_SAVE_PATH)

# Display the original data types
# print("Original Data Types:")
# print(data.dtypes)
# print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
data['is_rating_float'] = pd.to_numeric(data['few_shot_predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = data[data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
# # Re-run few-shot predictions for failed cases
# few_shot_data = pd.read_csv(FEW_SHOT_1_OBS_SAVE_PATH)
# # Add code to detect failed few-shot predictions similar to zero-shot
# few_shot_data['is_rating_float_few_shot'] = pd.to_numeric(few_shot_data['few_shot_predicted_rating'], errors='coerce').notna()

# # Assuming 'is_rating_float_few_shot' is the flag column for few-shot predictions
# failed_few_shot_indices = few_shot_data[few_shot_data['is_rating_float_few_shot'] == False].index
# few_shot_data = rerun_failed_predictions(
#     few_shot_data, 
#     failed_few_shot_indices, 
#     predict_ratings_few_shot_and_save, 
#     FEW_SHOT_1_OBS_SAVE_PATH,
#     columns_for_training=['title'],
#     columns_for_prediction=['title'],
#     obs_per_user=1,
#     pause_every_n_users=PAUSE_EVERY_N_USERS,
#     sleep_time=SLEEP_TIME
# )


In [ ]:
evaluate_model_predictions_rmse_mae(
    data_path=FEW_SHOT_1_OBS_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='few_shot_predicted_rating'
)


# Limitations:

The model might not fully understand the nuanced relationships between products based on titles alone. Additional context or features might be needed for more accurate predictions.
This approach might be computationally expensive and slower than traditional matrix factorization or deep learning-based recommendation models, especially for a large number of users.

# References

+ https://platform.openai.com/docs/api-reference/authentication